In [ ]:
### Market Environment Jupyter File

In [ ]:
!pip install keras
!pip install tensorflow

In [2]:
import os 

os.chdir(os.getcwd())
print(os.getcwd())

c:\Users\Nico\Documents\Uni\RL\Trading_Example


In [25]:
import numpy as np 
import pandas as pd 


class Market:
    def __init__(self, window_size, stock_name):
        self.data = self.get_stock_data(stock_name)
        self.states = self.get_all_window_prices_diff(self.data, window_size)
        self.index = -1
        self.last_data_index = len(self.data) - 1 

    def get_stock_data(self, stock_name):
        vec = []
        # creating list of lines - splitted
        lines = open(str(os.getcwd()) + "/data/" + stock_name + ".csv", "r").read().splitlines()
        for line in lines[1:]:
            vec.append(float(line.split(',')[4]))

        return vec

    def get_all_window_prices_diff(self, data, window_size):
        # Preprocsssing data to create list of states 

        processed_data = []

        for t in range(len(data)):
            state = self.get_window(data, t, window_size + 1)
            processed_data.append(state)
        
        return processed_data
    
    def get_window(self, data, t, n):
        # t - index of data
        # n - window size + 1 

        d = t - n + 1 # window block for each day 
        block = data[d:t+1] if d >= 0 else -d * [data[0]] + data[0:t+1]
        res = []
        for i in range(n-1):
            res.append(block[i+1]-block[i])
        
        return np.array([res])

    def reset(self):
        self.index = -1 
        return self.states[0], self.data[0]
    
    def get_next_state_reward(self, action, bought_price=None):
        # bought price only require for action = selling 

        self.index += 1 
        if self.index > self.last_data_index: # only checking 
            self.index = 0

        next_state = self.states[self.index +1]
        next_price_data = self.data[self.index + 1]

        price_data = self.data[self.index]
        reward = 0 
        # in case of selling - 2 // 0 holding // 1 buying 
        if action == 2 and bought_price is not None:
            reward = max(price_data - bought_price, 0) # just positive rewards 

        done = True if self.index == self.last_data_index - 1 else False 

        return next_state, next_price_data, reward, done 
    



In [29]:
market_1 = Market(3, "GSPC")

market_1.states

market_1.get_window(market_1.data, 2, n)

<bound method Market.get_window of <__main__.Market object at 0x000001EAD65E6130>>

In [34]:
from keras.models import Sequential 
from keras.models import load_model
from keras.layers import Dense
from keras.optimizers import Adam

import numpy as np
import random 
from collections import deque 


class Agent:
    def __init__(self, state_size, is_eval=False, model_name = ""):
        self.__inventory = []
        self.__total_profit = 0
        self.action_history = []

        self.state_size = state_size
        self.action_size = 3 # obvious 
        self.memory = deque(maxlen=1000)
        self.model_name = model_name
        self.is_eval = is_eval

        self.gamma = 0.95 
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995 # to geometrically decrease epislon until min value is reached
        

        self.model = load_model(os.getcwd()) + "/models/" + model_name) if is_eval else self.create_model()

        
        





ModuleNotFoundError: No module named 'tensorflow'